In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../data/data_interpolated.csv')
data.head() 

,TurbID,Day,Tmstamp,Wspd,Wdir,Etmp,Itmp,Ndir,Pab1,Pab2,Pab3,Prtv,Patv,Hour,Minute,T
0,1,1,00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,494.66,0,0,0
1,1,1,00:10,6.17,-3.99,30.73,41.80,25.92,1.0,1.0,1.0,-0.25,494.66,0,10,1
2,1,1,00:20,6.27,-2.18,30.60,41.63,20.91,1.0,1.0,1.0,-0.24,509.76,0,20,2
3,1,1,00:30,6.42,-0.73,30.52,41.52,20.91,1.0,1.0,1.0,-0.26,542.53,0,30,3
4,1,1,00:40,6.25,0.89,30.49,41.38,20.91,1.0,1.0,1.0,-0.23,509.36,0,40,4


In [3]:
def get_noisy_mask(df, p_noise):
    mask = np.random.rand(len(df)) < p_noise 

    return mask

mask = get_noisy_mask(data, 0.1)
noisy_data = data.copy()
noisy_data['Patv'] = noisy_data['Patv'].where(~mask, np.nan)

print('Entire dataset NaN %:', noisy_data['Patv'].isna().sum() / noisy_data.shape[0])
for idx, patv_series in noisy_data.groupby('TurbID')['Patv']:
    print(f'Turb {idx} NaN %: {patv_series.isna().sum() / len(patv_series)}')

Entire dataset NaN %: 0.10007678444512133
Turb 1 NaN %: 0.10025510204081632
Turb 2 NaN %: 0.09946145124716553
Turb 3 NaN %: 0.1007936507936508
Turb 4 NaN %: 0.10138888888888889
Turb 5 NaN %: 0.09866780045351474
Turb 6 NaN %: 0.10153061224489796
Turb 7 NaN %: 0.09914965986394558
Turb 8 NaN %: 0.1003968253968254
Turb 9 NaN %: 0.10578231292517007
Turb 10 NaN %: 0.09980158730158731
Turb 11 NaN %: 0.09914965986394558
Turb 12 NaN %: 0.09948979591836735
Turb 13 NaN %: 0.09861111111111111
Turb 14 NaN %: 0.10195578231292517
Turb 15 NaN %: 0.09946145124716553
Turb 16 NaN %: 0.10076530612244898
Turb 17 NaN %: 0.1016156462585034
Turb 18 NaN %: 0.10090702947845805
Turb 19 NaN %: 0.10113378684807256
Turb 20 NaN %: 0.09846938775510204
Turb 21 NaN %: 0.09997165532879819
Turb 22 NaN %: 0.1003968253968254
Turb 23 NaN %: 0.09974489795918368
Turb 24 NaN %: 0.10110544217687074
Turb 25 NaN %: 0.10158730158730159
Turb 26 NaN %: 0.1016156462585034
Turb 27 NaN %: 0.09923469387755102
Turb 28 NaN %: 0.0979591836

In [4]:
mask_df = pd.DataFrame(mask, columns=['NaN_Mask'])
mask_df.to_csv('../data/noise_mask.csv', index=False)

In [5]:
import numpy as np
import pandas as pd

def get_interval_noisy_mask(df, p_noise, noise_length, min_interval_between_noise = 6):
    mask = np.zeros(len(df), dtype=bool)

    # Iterate through each group (TurbID)
    for turb_id, group in df.groupby('TurbID'):
        group_len = len(group)
        rand_vals = np.random.rand(group_len)
        noisy_interval_starts = []
        
        i = 0
        while i < group_len:
            if rand_vals[i] < p_noise:
                if i + noise_length <= group_len:
                    noisy_interval_starts.append(i)
                    mask[group.index[i:i + noise_length]] = True
                    i += noise_length + min_interval_between_noise
                else:
                    i += 1
            else:
                i += 1

    return mask

mask = get_interval_noisy_mask(data, 0.1, 3)
interval_noisy_data = data.copy()
interval_noisy_data['Patv'] = interval_noisy_data['Patv'].where(~mask, np.nan)

print('entire dataset nan %:', interval_noisy_data['Patv'].isna().sum() / interval_noisy_data.shape[0])

for idx, patv_series in interval_noisy_data.groupby('TurbID')['Patv']:
    print('turb ' + str(idx) + ' nan %: ' + str(patv_series.isna().sum() / len(patv_series)))

entire dataset nan %: 0.1666006701187938
turb 1 nan %: 0.16437074829931972
turb 2 nan %: 0.16785714285714284
turb 3 nan %: 0.16972789115646258
turb 4 nan %: 0.166921768707483
turb 5 nan %: 0.1653061224489796
turb 6 nan %: 0.1691326530612245
turb 7 nan %: 0.16811224489795917
turb 8 nan %: 0.16454081632653061
turb 9 nan %: 0.1651360544217687
turb 10 nan %: 0.16437074829931972
turb 11 nan %: 0.1636904761904762
turb 12 nan %: 0.16845238095238096
turb 13 nan %: 0.16624149659863946
turb 14 nan %: 0.16462585034013605
turb 15 nan %: 0.16666666666666666
turb 16 nan %: 0.16666666666666666
turb 17 nan %: 0.16377551020408163
turb 18 nan %: 0.1635204081632653
turb 19 nan %: 0.16581632653061223
turb 20 nan %: 0.16488095238095238
turb 21 nan %: 0.16938775510204082
turb 22 nan %: 0.17270408163265305
turb 23 nan %: 0.16394557823129252
turb 24 nan %: 0.1655612244897959
turb 25 nan %: 0.1677721088435374
turb 26 nan %: 0.1677721088435374
turb 27 nan %: 0.16598639455782313
turb 28 nan %: 0.1645408163265306

In [6]:
mask_df = pd.DataFrame(mask, columns=['NaN_Mask'])
mask_df.to_csv('../data/interval_noise_mask.csv', index=False)